In [ ]:
import sys
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# Add the parent directory to path to import environments
sys.path.append(os.path.abspath('.'))
from environments.battleship_logic import BattleshipGame, BattleshipConfig

In [ ]:
N_GAMES = 1000
GRID_WIDTH = 10
GRID_HEIGHT = 10
SEED = 42

fleet_spec = {4: 1, 3: 2, 2: 3, 1: 4} # Standard Battleship config

In [ ]:
def check_win(game):
    # Check if all ships are sunk
    # 0=Water, 1..N=Ship IDs
    # shots: 2=Hit
    
    unique_ships = np.unique(game.ship_id_grid)
    for s_id in unique_ships:
        if s_id == 0: continue
        # Check if all cells for this ship ID are hit
        s_cells = (game.ship_id_grid == s_id)
        if not np.all(game.shots_grid[s_cells] == 2):
            return False
    return True

In [ ]:
class RandomAgent:
    def __init__(self, width, height, seed=None):
        self.width = width
        self.height = height
        self.rng = random.Random(seed)
        self.available_moves = [(r, c) for r in range(height) for c in range(width)]
        self.rng.shuffle(self.available_moves)
        
    def get_action(self):
        return self.available_moves.pop()

In [ ]:
results = []

print(f"Simulating {N_GAMES} games...")
for i in tqdm(range(N_GAMES)):
    # Initialize Game
    # Using a random seed for board generation for each game
    game = BattleshipGame(GRID_WIDTH, GRID_HEIGHT, fleet_spec, seed=None) 
    
    # Initialize Agent
    agent = RandomAgent(GRID_WIDTH, GRID_HEIGHT, seed=None)
    
    steps = 0
    while True:
        steps += 1
        r, c = agent.get_action()
        
        # Apply shot mechanism manually as per BattleshipGame logic
        if game.board[r, c] == 1:
            # It is a hit
            game.shots_grid[r, c] = 2 
        else:
            # It is a miss
            game.shots_grid[r, c] = 1 
            
        if check_win(game):
            results.append(steps)
            break

In [ ]:
print(f"Results for {N_GAMES} games trying to find all ships on {GRID_WIDTH}x{GRID_HEIGHT} grid:")
print(f"Average moves: {np.mean(results):.2f}")
print(f"Median moves: {np.median(results)}")
print(f"Min moves: {np.min(results)}")
print(f"Max moves: {np.max(results)}")

plt.figure(figsize=(10, 6))
plt.hist(results, bins=range(0, GRID_WIDTH * GRID_HEIGHT + 2, 2), alpha=0.7, color='skyblue', edgecolor='black')
plt.title(f"Distribution of Moves to Win (Random Agent)")
plt.xlabel("Number of Moves")
plt.ylabel("Frequency")
plt.grid(axis='y', alpha=0.5)
plt.axvline(np.mean(results), color='red', linestyle='dashed', linewidth=1, label=f'Mean: {np.mean(results):.2f}')
plt.legend()
plt.show()